In [97]:
import numpy as np
import math

e = 2.718281828459045
# rates tree 
"""
* for intial term structure P(0, t), given inputs {(T_i, P(0,T_i))}, get yeild curve and apply linear interpolation
* implement Hull-White one factor model, and Black-Karasinski model
* use ternary tree branching mechanism, three kinds of branching
* store whole tree as list of lists (better in struct datatype), instead of one level (doable but less efficient)
"""

class Ternary_tree:
    def __init__(self, a, sigma, T, depth, init_term_struct, rate_model="Hull-White"):
        """ a: a factor in Hull-White or Black-Karasinski model
            sigma: sigma factor in model
            T: end time of the tree
            depth: tree branching times
            init_term_struct: initial zero coupon bonds in form {(T_i, P(0, T_i))}
            rate_model: currently support Hull-White and Black-Karasinski rate models
        """
        self.a = a
        self.sigma = sigma
        self.T = T
        self.depth = depth
        self.init_term_struct = init_term_struct
        self.rate_model = rate_model
        
        self.dt = T/max(1, self.depth)
        self.dx = self.sigma*(3*self.dt)**0.5
        self.rate_from_x = self.get_rate(self.rate_model)
        
        self.tree = None
        
        self.P_0i = None
        
        self.alpha_precision = 1.e-6
        self.alpha_lohi = None
               
    
    def build_tree(self):
        a, sigma, T, depth = self.a, self.sigma, self.T, self.depth
        dt, dx = self.dt, self.dx
        rate_from_x = self.rate_from_x
        
        self.calculate_init_zero_coupons()
        
        max_radius = int(np.ceil(0.184/a/dt))

        self.tree = [[{"Q":1, "probs":[None, None, None], "prob": 1}]]
        
        for lvl in range(depth):
            self.tree.append([{"Q":0, "probs":[None, None, None], "prob":0}\
                              for _ in range(2*min(lvl+1, max_radius)+1)])
            
            alpha = self.get_alpha(lvl)
            
            # Can add a variable nodes = self.tree[lvl], to simplify notation.
            for j in range(-min(lvl, max_radius-1), min(lvl, max_radius-1)+1):
                self.tree[lvl][j]["rate"] = rate_from_x(j*dx+alpha)
                
                self.tree[lvl][j]["probs"][0] = 1/6+0.5*(a*a*j*j*dt*dt+a*j*dt)
                self.tree[lvl][j]["probs"][1] = 2/3-a*a*j*j*dt*dt
                self.tree[lvl][j]["probs"][2] = 1/6+0.5*(a*a*j*j*dt*dt-a*j*dt)
                
                for jk in range(3):
                    self.tree[lvl+1][j+jk-1]["Q"] += self.tree[lvl][j]["probs"][jk]*self.tree[lvl][j]["Q"]\
                                                    *np.e**(-rate_from_x(alpha+j*dx)*dt)
                    self.tree[lvl+1][j+jk-1]["prob"] += self.tree[lvl][j]["probs"][jk]*self.tree[lvl][j]["prob"]
                
            if lvl >= max_radius:
                for sign in (-1, 1):
                    j_b = max_radius*sign
                    self.tree[lvl][j_b]["rate"] = rate_from_x(j_b*dx+alpha)
                    
                    self.tree[lvl][j_b]["probs"][1+sign] = 7/6+0.5*(a*a*j_b*j_b*dt*dt-3*sign*a*j_b*dt)
                    self.tree[lvl][j_b]["probs"][1] = -1/3-a*a*j_b*j_b*dt*dt+2*sign*a*j_b*dt
                    self.tree[lvl][j_b]["probs"][1-sign] = 1/6+0.5*(a*a*j_b*j_b*dt*dt-sign*a*j_b*dt)
                    
                    for jk in range(3):
                        self.tree[lvl+1][j_b+jk-sign-1]["Q"] += self.tree[lvl][j_b]["probs"][jk]\
                                  *self.tree[lvl][j_b]["Q"]*np.e**(-rate_from_x(alpha+j_b*dx)*dt)
                        self.tree[lvl+1][j_b+jk-sign-1]["prob"] += self.tree[lvl][j_b]["probs"][jk]\
                                  *self.tree[lvl][j_b]["prob"]
        
        lvl += 1
        alpha = self.get_alpha(lvl)
        for j in range(-min(lvl, max_radius), min(lvl, max_radius)+1):
            self.tree[lvl][j]["rate"] = rate_from_x(j*dx+alpha)
            
        return 
    
    def future_bonds(self):
        dt = self.dt
        N = self.depth
        max_radius = int(np.ceil(0.184/self.a/self.dt))
        
        if self.tree == None:
            self.build_tree
        
        for j in range(len(self.tree[-1])):
            self.tree[-1][j]["P_tiT"] = 1
        
        for lvl in range(N-1, -1, -1):
            for j in range(-min(lvl, max_radius-1), min(lvl, max_radius-1)+1):
                self.tree[lvl][j]["P_tiT"] = 0
                for jk in range(3):
                    self.tree[lvl][j]["P_tiT"] += self.tree[lvl][j]["probs"][jk]\
                                                  *self.tree[lvl+1][j+jk-1]["P_tiT"]
                self.tree[lvl][j]["P_tiT"] *= np.e**(-dt*self.tree[lvl][j]["rate"])
                
            if lvl >= max_radius:
                for sign in (-1, 1):
                    j_b = max_radius*sign
                    self.tree[lvl][j_b]["P_tiT"] = 0
                    for jk in range(3):
                        self.tree[lvl][j_b]["P_tiT"] += self.tree[lvl][j_b]["probs"][jk]\
                                                        *self.tree[lvl+1][j_b+jk-1-sign]["P_tiT"]
                    self.tree[lvl][j_b]["P_tiT"] *= np.e**(-dt*self.tree[lvl][j_b]["rate"])
        
        return
        


    def calculate_init_zero_coupons(self):
        dt = self.dt
        N = self.depth+2
        bonds = sorted(self.init_term_struct, key=lambda x:x[0])
        
        if len(bonds) == 1:
            y_flat = -np.log(bonds[0][1])/bonds[0][0]
            self.P_0i = [e**(-i*dt*y_flat) for i in range(N)]
            return 
        
        yields = [[bonds[i][0], -np.log(bonds[i][1])/bonds[i][0]] for i in range(len(bonds))]
        T_r_ext = max(yields[-1][0], self.T) + 2*dt
        y_r_ext = (yields[-1][1]-yields[-2][1])/(yields[-1][0]-yields[-2][0])\
                  *(T_r_ext-yields[-1][0])+yields[-1][1]
        T_l_ext = 0
        y_l_ext = (yields[1][1]-yields[0][1])/(yields[1][0]-yields[0][0])*(T_l_ext-yields[0][0])+yields[0][1]
        y_l_ext = max(y_l_ext, 0)
        yields = [[T_l_ext, y_l_ext]] + yields + [[T_r_ext, y_r_ext]]
        
        self.P_0i = [1]
        p = 0
        for i in range(1, N):
            ti = i*dt
            while ti > yields[p][0]:
                p += 1
            y_i = (yields[p][1]-yields[p-1][1])/(yields[p][0]-yields[p-1][0])*(ti-yields[p-1][0])+yields[p-1][1]
            self.P_0i.append(np.e**(-ti*y_i))
        
        return 
        
    
    def get_rate(self, model_name):
        model_maps = dict()
        model_maps["Hull-White"] = lambda x: x
        model_maps["Black-Karasinski"] = lambda x: np.e**x
        
        if model_name not in model_maps:
            print("rate model not supported")
            exit(1)
        
        return model_maps[model_name]
       
        
    def get_alpha(self, lvl):
        # can be optimized by using interpolation properly.
        alpha = None
        dt, dx = self.dt, self.dx
        radius = round((len(self.tree[lvl])-1)/2)
        
        P_eval = lambda x:sum([self.tree[lvl][j]["Q"]*np.e**(-self.rate_from_x(x+j*dx)*dt)\
                                  for j in range(-radius, radius+1)])
        
        if self.alpha_lohi == None:
            self.update_alpha_binary_lohi()
        lo, hi = self.alpha_lohi  
        
        while hi-lo > self.alpha_precision:
            alpha = (hi+lo)/2
            if P_eval(alpha) > self.P_0i[lvl+1]:
                lo = alpha
            else:
                hi = alpha

        return alpha
    
    
    def update_alpha_binary_lohi(self):
        lohi_maps = dict()
        lohi_maps["Hull-White"] = [0.01, 0.9]
        lohi_maps["Black-Karasinski"] = [-10, 0]
        
        self.alpha_lohi = lohi_maps[self.rate_model]
        
        return 

def yield_curve(P_0i):
    bonds = list(sorted(P_0i, key=lambda x:x[0]))
    
    if len(bonds) == 1:
        y_flat = -np.log(bonds[0][1])/bonds[0][0]
        return [[0, y_flat], [P_0i[0], y_flat], [P_0i[0]+10, y_flat]]

    yields = [[bonds[i][0], -np.log(bonds[i][1])/bonds[i][0]] for i in range(len(bonds))]
    T_r_ext = yields[-1][0] + 10
    y_r_ext = (yields[-1][1]-yields[-2][1])/(yields[-1][0]-yields[-2][0])*(T_r_ext-yields[-1][0]) + yields[-1][1]
    T_l_ext = 0
    y_l_ext = (yields[1][1]-yields[0][1])/(yields[1][0]-yields[0][0])*(T_l_ext-yields[0][0]) + yields[0][1]
    y_l_ext = max(y_l_ext, 0)
    yields = [[T_l_ext, y_l_ext]] + yields + [[T_r_ext, y_r_ext]]

    return yields

In [98]:
""" Test example 31-4 .
    Method 1.
    Build a rates tree from t=0 to t=T_ast=9.
    Future bond is P(T=3, T_ast=9), use the tree to calculate option price on bond.
"""

spot_rates = [[3/365, 0.0501722], [31/365, 0.0498284], [62/365, 0.0497234], [94/365, 0.0496157],\
              [185/365, 0.0499058], [367/365, 0.0509389], [731/365, 0.0579733], [1096/365, 0.0630595], \
              [1461/365, 0.0673464], [1826/365, 0.0694816], [2194/365, 0.0708807], [2558/365, 0.0727527], \
              [2922/365, 0.0730852], [3287/365, 0.0739790], [3653/365, 0.0749015]]
P_0i = [[row[0], np.e**(-row[0]*row[1])] for row in spot_rates]

a, sigma, T, T_ast, depth = 0.1, 0.01, 3, 9, 500
normal_model, log_normal_model = "Hull-White", "Black-Karasinski"
K, L = 63, 100

def opt_price(rates_tree):
    dt =T_ast/depth
    i_l = math.floor(T/dt)
    i_r = i_l+1
    
    c_l, c_r = 0, 0
    p_l, p_r = 0, 0
    for node in rates_tree[i_l]:
        c_l += node["Q"]*max(L*node["P_tiT"]-K, 0)
        p_l += node["Q"]*max(K-L*node["P_tiT"], 0)
    for node in tree[i_r]:
        c_r += node["Q"]*max(L*node["P_tiT"]-K, 0)
        p_r += node["Q"]*max(K-L*node["P_tiT"], 0)
    
    c = (c_r-c_l)/dt*(T-i_l*dt)+c_l
    p = (p_r-p_l)/dt*(T-i_l*dt)+p_l
    return c, p

rates_tree = Ternary_tree(a, sigma, T_ast, depth, P_0i, rate_model=normal_model)
rates_tree.alpha_precision = 1.e-7
rates_tree.build_tree()
rates_tree.future_bonds()
tree = rates_tree.tree

c, p = opt_price(tree)
c, p

(1.0559743267629855, 1.8117334943101002)

In [99]:
""" Test example 31-4 .
    Method 2.
    Build a rates tree from t=0 to t=T=3.
    Use analytical future bond prices on the end level of the tree.
    Then use Qs to calculate option price on the bond.
"""

# spot_rates = [[0.5, 0.03430], [1, 0.03824], [1.5, 0.04183], [2, 0.04512], [2.5, 0.04812], [3, 0.05086]]

spot_rates = [[3/365, 0.0501722], [31/365, 0.0498284], [62/365, 0.0497234], [94/365, 0.0496157],\
              [185/365, 0.0499058], [367/365, 0.0509389], [731/365, 0.0579733], [1096/365, 0.0630595], \
              [1461/365, 0.0673464], [1826/365, 0.0694816], [2194/365, 0.0708807], [2558/365, 0.0727527], \
              [2922/365, 0.0730852], [3287/365, 0.0739790], [3653/365, 0.0749015]]
P_0i = [[row[0], e**(-row[0]*row[1])] for row in spot_rates]
ys = yield_curve(P_0i)


a, sigma, T, T_ast, depth = 0.1, 0.01, 3, 9, 200
normal_model, log_normal_model = "Hull-White", "Black-Karasinski"
# rates_tree = Ternary_tree(a, sigma, T_ast, depth, P_0i, rate_model=normal_model)
# rates_tree.alpha_precision = 1.e-6
# rates_tree.build_tree()

def P_0t(t):
    p = 0
    while t > ys[p][0]:
        p += 1    
    y = (ys[p][1]-ys[p-1][1])/(ys[p][0]-ys[p-1][0])*(t-ys[p-1][0])+ys[p-1][1] 
    return e**(-t*y)

def P_tT(R, dt, t, T):
    B = lambda x, y: (1-e**(-a*(y-x)))/a
    A_hat = P_0t(T)/P_0t(t)*(P_0t(t)/P_0t(t+dt))**(B(t,T)/B(t,t+dt))
    A_hat *= e**(sigma*sigma/4/a*(e**(-2*a*t)-1)*B(t,T)*(B(t,T)-B(t,t+dt)))
    return A_hat*e**(-R*dt*B(t,T)/B(t,t+dt))

def euro_option(tree_depth, tree_model, K):
    rates_tree = Ternary_tree(a, sigma, T, tree_depth, P_0i, rate_model=tree_model)
    rates_tree.build_tree()
    #print(rates_tree.tree[-1])
    call, put = 0, 0
    for node in rates_tree.tree[-1]:
        bond_price = 100*P_tT(node["rate"], T/tree_depth, T, T_ast)
        #print(bond_price, node["prob"], node["Q"])
        call += node["Q"]*max(bond_price-K, 0)
        put += node["Q"]*max(K-bond_price, 0)
    return (call, put)

tree_depth = 100
K = 63
tree_model = normal_model
call, put = euro_option(tree_depth, tree_model, K)
print("call, put: ", call, put)

call, put:  1.0596857287654002 1.8143293993370044


In [ ]:
""" Try to get proper parameters for log_normal model.
"""
min_loss = float("inf")
a0, sigma0 = None, None
c0, p0 = None, None
for i in range(100):
    a = 0.001 + 0.0004*i
    for j in range(50):
        sigma = 0.01 + 0.0004*j
        rates_tree = Ternary_tree(a, sigma, T_ast, depth, P_0i, rate_model=log_normal_model)
        rates_tree.alpha_precision = 1.e-7
        rates_tree.build_tree()
        rates_tree.future_bonds()
        tree = rates_tree.tree
        call, put = opt_price(tree)
        loss = (call-1.0559743267629855)**2+(put-1.8117334943101002)**2
        if loss < min_loss:
            min_loss = loss
            c0, p0 = call, put
            a0, sigma0 = a, sigma
    if i%3 == 0:
        print("i = ", i)
        print("  loss, call, put", min_loss, c0, p0)
        print("  a, sigma: ", a0, sigma0)

c, p = opt_price(tree)
print(c, p)